# Kaggle data download

In [ ]:
!pip install kaggle
from google.colab import files
files.upload()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"jeonhyotaek","key":"d2ddaf4c586e0bf63051e1a4c6a74dd4"}'}

In [ ]:
ls -1ha kaggle.json

kaggle.json


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle competitions download -c copy-of-6th-goorm-project-1-text-classification

  0% 0.00/6.12M [00:00<?, ?B/s]
100% 6.12M/6.12M [00:00<00:00, 94.6MB/s]


In [ ]:
!unzip copy-of-6th-goorm-project-1-text-classification.zip

Archive:  copy-of-6th-goorm-project-1-text-classification.zip
  inflating: sentiment.dev.0         
  inflating: sentiment.dev.1         
  inflating: sentiment.train.0       
  inflating: sentiment.train.1       
  inflating: test_no_label.csv       


# Import requirements

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 5.0 MB/s 
     |████████████████████████████████| 182 kB 96.6 MB/s 
     |████████████████████████████████| 7.6 MB 83.1 MB/s 


In [ ]:
import os
import pdb
import argparse
from dataclasses import dataclass, field
from typing import Optional
from collections import defaultdict

import torch
from torch.nn.utils.rnn import pad_sequence

import numpy as np
from tqdm import tqdm, trange

from transformers import (
    BertForSequenceClassification,
    BertTokenizer,
    AutoConfig,
    AdamW
)

# Preprocess

In [ ]:
def make_id_file(task, tokenizer):
    def make_data_strings(file_name):
        data_strings = []
        with open(os.path.join(file_name), 'r', encoding='utf-8') as f:
            id_file_data = [tokenizer.encode(line.lower()) for line in f.readlines()]
        for item in id_file_data:
            data_strings.append(' '.join([str(k) for k in item]))
        return data_strings
    
    print('it will take some times...')
    train_pos = make_data_strings('sentiment.train.1')
    train_neg = make_data_strings('sentiment.train.0')
    dev_pos = make_data_strings('sentiment.dev.1')
    dev_neg = make_data_strings('sentiment.dev.0')

    print('make id file finished!')
    return train_pos, train_neg, dev_pos, dev_neg

In [65]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
!ls

copy-of-6th-goorm-project-1-text-classification.zip  sentiment.dev.1
kaggle.json					     sentiment.train.0
sample_data					     sentiment.train.1
sentiment.dev.0					     test_no_label.csv


In [ ]:
train_pos, train_neg, dev_pos, dev_neg = make_id_file('yelp', tokenizer)

it will take some times...
make id file finished!


In [ ]:
train_pos[:10]

['101 6581 2833 1012 102',
 '101 21688 8013 2326 1012 102',
 '101 2027 2036 2031 3679 19247 1998 3256 6949 2029 2003 2428 2204 1012 102',
 '101 2009 1005 1055 1037 2204 15174 2098 7570 22974 2063 1012 102',
 '101 1996 3095 2003 5379 1012 102',
 '101 2204 3347 2833 1012 102',
 '101 2204 2326 1012 102',
 '101 11350 1997 2154 2003 25628 1998 7167 1997 19247 1012 102',
 '101 2307 2173 2005 6265 2030 3347 27962 1998 5404 1012 102',
 '101 1996 2047 2846 3504 6429 1012 102']

In [ ]:
class SentimentDataset(object):
    def __init__(self, tokenizer, pos, neg):
        self.tokenizer = tokenizer
        self.data = []
        self.label = []

        for pos_sent in pos:
            self.data += [self._cast_to_int(pos_sent.strip().split())]
            self.label += [[1]]
        for neg_sent in neg:
            self.data += [self._cast_to_int(neg_sent.strip().split())]
            self.label += [[0]]

    def _cast_to_int(self, sample):
        return [int(word_id) for word_id in sample]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        sample = self.data[index]
        return np.array(sample), np.array(self.label[index])

In [ ]:
train_dataset = SentimentDataset(tokenizer, train_pos, train_neg)
dev_dataset = SentimentDataset(tokenizer, dev_pos, dev_neg)

In [ ]:
for i, item in enumerate(train_dataset):
    print(item)
    if i == 10:
        break

(array([ 101, 6581, 2833, 1012,  102]), array([1]))
(array([  101, 21688,  8013,  2326,  1012,   102]), array([1]))
(array([  101,  2027,  2036,  2031,  3679, 19247,  1998,  3256,  6949,
        2029,  2003,  2428,  2204,  1012,   102]), array([1]))
(array([  101,  2009,  1005,  1055,  1037,  2204, 15174,  2098,  7570,
       22974,  2063,  1012,   102]), array([1]))
(array([ 101, 1996, 3095, 2003, 5379, 1012,  102]), array([1]))
(array([ 101, 2204, 3347, 2833, 1012,  102]), array([1]))
(array([ 101, 2204, 2326, 1012,  102]), array([1]))
(array([  101, 11350,  1997,  2154,  2003, 25628,  1998,  7167,  1997,
       19247,  1012,   102]), array([1]))
(array([  101,  2307,  2173,  2005,  6265,  2030,  3347, 27962,  1998,
        5404,  1012,   102]), array([1]))
(array([ 101, 1996, 2047, 2846, 3504, 6429, 1012,  102]), array([1]))
(array([ 101, 2023, 2173, 2001, 2200, 2204, 1012,  102]), array([1]))


In [ ]:
def collate_fn_style(samples):
    input_ids, labels = zip(*samples)
    max_len = max(len(input_id) for input_id in input_ids)

    sorted_indices = np.argsort([len(input_id) for input_id in input_ids])[::-1]

    input_ids = pad_sequence([torch.tensor(input_ids[index]) for index in sorted_indices],
                             batch_first=True)

    attention_mask = torch.tensor(
        [[1] * len(input_ids[index]) + [0] * (max_len - len(input_ids[index])) for index in
         sorted_indices])
    token_type_ids = torch.tensor([[0] * len(input_ids[index]) for index in sorted_indices])
    position_ids = torch.tensor([list(range(len(input_ids[index]))) for index in sorted_indices])
    labels = torch.tensor(np.stack(labels, axis=0)[sorted_indices])

    return input_ids, attention_mask, token_type_ids, position_ids, labels

# Wandb sweep

In [71]:
!pip install wandb -qqq
import wandb
wandb.login()

True

In [102]:
sweep_config = {
      'name' : 'goorm_project1_test.ipynb',
      'method' : 'grid',
      'metric':{
          'name': 'validation_acc',
          'goal': 'maximize'  
      },
      'parameters' : {
          'learning_rate' : {
              'values' : [5e-5]
          },   
          'batch_size' :{
              'values' : [512]
          },
          'epochs' : {
              'values' : [1] 
          }
      }
}

sweep_id = wandb.sweep(sweep_config)

Create sweep with ID: cteknfiz
Sweep URL: https://wandb.ai/goorm_project/uncategorized/sweeps/cteknfiz


# Train model def

In [99]:
def sweep_optimizer(input_model, optimizer, learning_rate) :
    if optimizer == "AdamW" :
        optimizer = torch.optim.AdamW(input_model.parameters(), lr = learning_rate, eps = 1e-6, weight_decay = 0.02)
    elif optimizer == 'RMSprop' :
        optimizer = torch.optim.RMSprop(input_model.parameters(), lr = learning_rate, weight_decay = 0.02)
    return optimizer

In [103]:
def train():
    wandb.init(project = 'test_sweep', reinit = True)

    ## batch_size - sweep
    train_batch_size = int(wandb.config.batch_size / 2)
    eval_batch_size = wandb.config.batch_size

    # dataload
    train_loader = torch.utils.data.DataLoader(train_dataset,
                                              batch_size=train_batch_size,
                                              shuffle=True, collate_fn=collate_fn_style,
                                              pin_memory=True, num_workers=2)
    dev_loader = torch.utils.data.DataLoader(dev_dataset, batch_size=eval_batch_size,
                                            shuffle=False, collate_fn=collate_fn_style,
                                            num_workers=2)
    ## random seed
    random_seed=42
    np.random.seed(random_seed)
    torch.manual_seed(random_seed)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # model-pretrain
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
    model.to(device)

    # model train - sweep
    model.train()
    learning_rate = wandb.config.learning_rate
    optimizer =  AdamW(model.parameters(), lr=learning_rate, eps = 1e-6, weight_decay = 0.02)

    def compute_acc(predictions, target_labels):
        return (np.array(predictions) == np.array(target_labels)).mean()

    # train_epoch
    train_epoch = wandb.config.epochs
    # validation_acc
    wan_valid_acc = []

    lowest_valid_loss = 9999.

    #train
    for epoch in range(train_epoch):
        #total train loss
        wan_train_loss = []

        with tqdm(train_loader, unit="batch") as tepoch:
            
            for iteration, (input_ids, attention_mask, token_type_ids, position_ids, labels) in enumerate(tepoch):
                tepoch.set_description(f"Epoch {epoch}")
                input_ids = input_ids.to(device)
                attention_mask = attention_mask.to(device)
                token_type_ids = token_type_ids.to(device)
                position_ids = position_ids.to(device)
                labels = labels.to(device, dtype=torch.long)

                optimizer.zero_grad()

                output = model(input_ids=input_ids,
                              attention_mask=attention_mask,
                              token_type_ids=token_type_ids,
                              position_ids=position_ids,
                              labels=labels)

                loss = output.loss
                loss.backward()

                # log the train loss / total loss
                wandb.log({'train_batch_loss':loss.item()})
                wan_train_loss.append(loss.item())

                optimizer.step()

                tepoch.set_postfix(loss=loss.item())

                #sum_acc
                sum_acc = [] 

                if iteration != 0 and iteration % int(len(train_loader) / 5) == 0:
                    # Evaluate the model five times per epoch
                    with torch.no_grad():
                        model.eval()
                        valid_losses = []
                        predictions = []
                        target_labels = []
                        for input_ids, attention_mask, token_type_ids, position_ids, labels in tqdm(dev_loader,
                                                                                                    desc='Eval',
                                                                                                    position=1,
                                                                                                    leave=None):
                            input_ids = input_ids.to(device)
                            attention_mask = attention_mask.to(device)
                            token_type_ids = token_type_ids.to(device)
                            position_ids = position_ids.to(device)
                            labels = labels.to(device, dtype=torch.long)

                            output = model(input_ids=input_ids,
                                          attention_mask=attention_mask,
                                          token_type_ids=token_type_ids,
                                          position_ids=position_ids,
                                          labels=labels)

                            logits = output.logits
                            loss = output.loss
                            valid_losses.append(loss.item())

                            batch_predictions = [0 if example[0] > example[1] else 1 for example in logits]
                            batch_labels = [int(example) for example in labels]

                            predictions += batch_predictions
                            target_labels += batch_labels


                    acc = compute_acc(predictions, target_labels)
                    valid_loss = sum(valid_losses) / len(valid_losses)

                    wan_valid_acc.append(float(acc))
                    wandb.log({'validation_acc': sum(wan_valid_acc)/len(wan_valid_acc)})

                    sum_acc.append(float(acc)) ############
                    wandb.log({"sum_acc": sum(sum_acc) / len(sum_acc)}) #####


                    if lowest_valid_loss > valid_loss:
                        print('Acc for model which have lower valid loss: ', acc)
                        torch.save(model.state_dict(), "./pytorch_model.bin")
                        lowest_valid_loss = valid_loss

                else:
                    #avg train loss
                    wandb.log({'avg_train_loss': sum(wan_train_loss) / len(wan_train_loss)})


In [104]:
wandb.agent( sweep_id , function=train, count=1)

wandb: Agent Starting Run: 6akwvf2w with config:
wandb: 	batch_size: 512
wandb: 	epochs: 1
wandb: 	learning_rate: 5e-05
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Acc for model which have lower valid loss:  0.972


Epoch 0:  27%|██▋       | 462/1732 [01:50<04:52,  4.35batch/s, loss=0.0661]wandb: Ctrl + C detected. Stopping sweep.


In [ ]:
if __name__ == '__main__':
    main()